In [1]:
%cd ../

/home/qwj/code/HippoRAG


In [2]:
import subprocess

In [3]:
import os
import shutil

def is_new_files(file_name):
    return not (file_name.startswith("openie_") or file_name.startswith('.') or file_name.endswith("named_entity_output.tsv") or file_name.endswith("entity_and_relations.tsv"))

def move_files(subfolder_name):
    # 定义新子文件夹的名称
    new_subfolder = os.path.join('output', subfolder_name)
    
    # 如果新子文件夹不存在，创建它
    if not os.path.exists(new_subfolder):
        os.makedirs(new_subfolder)

    # 获取文件夹下的所有文件（不包含子文件夹）
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)
        
        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                # 移动文件到新子文件夹
                print(f"Moved: {file_name}")
                shutil.move(file_path, os.path.join(new_subfolder, file_name))

def show_new_files():
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)
    
        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                print(file_name)

def remove_new_files():
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)

        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                print(f"Removed: {file_name}")
                os.remove(file_path)


## Step 1 Create Retriever Corpus 

In [6]:
!python src/create_graph_v2.py \
    --dataset musique \
    --model_name colbertv2 \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.8 \
    --extraction_type ner

100%|██████████████████████████████████| 11656/11656 [00:01<00:00, 10142.54it/s]
Correct Wiki Format: 0 out of 11656


In [7]:
show_new_files()

query_to_kb.tsv
rel_kb_to_kb.tsv
musique_queries.entity_and_relations.tsv
kb_to_kb.tsv


## compute similar edges

### kb internal similarity

here we have entities and relations

In [2]:
!CUDA_VISIBLE_DEVICES="4" python src/colbertv2_knn.py --filename output/kb_to_kb.tsv



[Jul 05, 04:03:29] #> Creating directory colbert/indexes/nbits_2 


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 180,
    "mask_punctuation": true,
    "checkpoint": "exp\/colbert

### query to kb similarity

In [ ]:
!CUDA_VISIBLE_DEVICES="2,3" python src/colbertv2_knn.py --filename output/query_to_kb.tsv

## Create Real Graph

In [6]:
remove_new_files()

Removed: musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_similarity_edges_mean_0.8_thresh_ents_only_lower_preprocess_ner_colbertv2.v3.subset.p
Removed: musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: query_to_kb.tsv
Removed: rel_kb_to_kb.tsv
Removed: musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_passage_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_facts_and_sim_graph_fact_doc_edges_ents_only_lower_preprocess_ner.v3.subset.p
Removed: kb_to_kb.tsv
Removed: musique_facts_and_sim_graph_doc_to_facts_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_doc_to_facts_csr_ents_only_lower_preprocess_ner.v3.subset.p
Removed: musique_facts_and_sim_graph_

In [4]:
!python src/create_graph_v2.py \
    --dataset musique \
    --model_name colbertv2 \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.8 \
    --extraction_type ner \
    --create_graph \
    --cosine_sim_edges

100%|██████████████████████████████████| 11656/11656 [00:01<00:00, 10231.22it/s]
Correct Wiki Format: 0 out of 11656
Creating Graph
100%|██████████████████████████████████| 11656/11656 [00:00<00:00, 38181.91it/s]
Loading Vectors
Augmenting Graph from Similarity
100%|████████████████████████████████| 116708/116708 [00:01<00:00, 87846.65it/s]
Saving Graph
                                                         1
0                                                         
Total Phrases                                       327435
Unique Phrases                                      113611
Number of Individual Triples                        109145
Number of Incorrectly Formatted Triples (ChatGP...    1136
Number of Triples w/o NER Entities (ChatGPT Error)    6762
Number of Unique Individual Triples                 107448
Number of Entities                                  218290
Number of Relations                                 239470
Number of Unique Entities                            9

## Index

In [5]:
!CUDA_VISIBLE_DEVICES="0" python3 src/colbertv2_indexing.py \
    --dataset musique \
    --phrase output/musique_facts_and_sim_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p \
    --corpus data/musique_corpus.json



[Jul 05, 04:16:20] #> Creating directory data/lm_vectors/colbert/musique/corpus/indexes/exp/colbertv2.0 


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 180,
    "mask_punctuation"

---

## Contriever

In [5]:
remove_new_files()

Removed: musique_facts_and_sim_graph_mean_0.6_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_clean_facts_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: musique_similarity_edges_mean_0.575_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_nodes_chatgpt_openIE.ents_only_lower_preprocess_ner.v3.subset.json
Removed: query_to_kb.tsv
Removed: musique_facts_and_sim_graph_mean_0.475_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_mean_0.575_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_mean_0.55_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_sim_graph_mean_0.525_thresh_ents_only_lower_preprocess_ner_grit-7b.v3.subset.p
Removed: musique_facts_and_si

In [6]:
!python src/create_graph_v2.py \
    --dataset musique \
    --model_name facebook/contriever \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.8 \
    --extraction_type ner

100%|██████████████████████████████████| 11656/11656 [00:01<00:00, 10070.51it/s]
Correct Wiki Format: 0 out of 11656


In [7]:
show_new_files()

query_to_kb.tsv
rel_kb_to_kb.tsv
kb_to_kb.tsv


### INDEX

In [9]:

# 定义三个命令和对应的CUDA设备
commands = [
    ("CUDA_VISIBLE_DEVICES=0 python src/RetrievalModule.py --retriever_name facebook/contriever --string_filename output/query_to_kb.tsv", 0),
    ("CUDA_VISIBLE_DEVICES=1 python src/RetrievalModule.py --retriever_name facebook/contriever --string_filename output/rel_kb_to_kb.tsv", 1),
    ("CUDA_VISIBLE_DEVICES=2 python src/RetrievalModule.py --retriever_name facebook/contriever --string_filename output/kb_to_kb.tsv", 2),
]

# 启动每个命令
processes = []
for command, cuda_device in commands:
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = str(cuda_device)
    process = subprocess.Popen(command, shell=True, env=env)
    processes.append(process)

# 等待所有进程完成
for process in processes:
    process.wait()

print("All processes completed.")


No Pre-Computed Vectors. Confirming PLM Model.
No Pre-Computed Vectors. Confirming PLM Model.
No Pre-Computed Vectors. Confirming PLM Model.
Loading PLM Vectors.
Loading PLM Vectors.
Loading PLM Vectors.


100%|██████████| 116432/116432 [00:00<00:00, 2794714.48it/s]


Encoding 502 Missing Strings
Encoding 1635 Missing Strings


100%|██████████| 227222/227222 [00:00<00:00, 2716183.43it/s]


Encoding 502 Missing Strings


...

Running Index Part 3



100%|██████████| 100/100 [00:02<00:00, 36.18it/s]


Combining Index Chunks


113611it [00:17, 6660.62it/s]
113611it [00:30, 3743.65it/s]


All processes completed.


In [6]:
import pandas as pd


def calculate_recall_mean(output_path):
    result = pd.read_json(output_path)
    def recall_dict_to_columns(recall_dict):
        return pd.Series({f'recall@{k}': v for k, v in recall_dict.items()})
    recall_df = result['recall'].apply(recall_dict_to_columns)
    return recall_df.mean()

In [4]:
# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

create_graph_command = [
    'python3', 'src/create_graph_v2.py',
    '--dataset', 'musique',
    '--model_name', 'facebook/contriever',
    '--extraction_model', 'gpt-3.5-turbo-1106',
    '--threshold', "0.8",
    '--extraction_type', 'ner',
    '--create_graph', '--cosine_sim_edges'
]
subprocess.run(create_graph_command, check=True)

ircot_hipporag_command = [
    'python3', 'src/ircot_hipporag_v2.py',
    '--dataset', 'musique',
    '--retriever', 'facebook/contriever',
    '--max_steps', '1',
    '--doc_ensemble', 'f',
    '--top_k', '10',
    '--sim_threshold', "0.8",
    '--damping', '0.5',
]
subprocess.run(ircot_hipporag_command, check=True)


100%|██████████| 11656/11656 [00:01<00:00, 10427.83it/s]


Correct Wiki Format: 0 out of 11656
Creating Graph


100%|██████████| 11656/11656 [00:00<00:00, 37551.52it/s]


Loading Vectors
Augmenting Graph from Similarity


100%|██████████| 113611/113611 [00:03<00:00, 31457.80it/s]


...

Building Graph: 100%|██████████| 585007/585007 [00:01<00:00, 579614.79it/s]
2024-07-12 15:39:15,749 - hipporag_v2 - INFO - Graph built: num vertices: 113611, num_edges: 584896
2024-07-12 15:39:15,794 - hipporag_v2 - INFO - Loading node vectors from: data/lm_vectors/facebook_contriever_mean/encoded_strings.txt
2024-07-12 15:39:16,503 - hipporag_v2 - INFO - 1 phrases did not have vectors.
IRCoT retrieval:   0%|          | 0/1000 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'output/ircot_v2/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.8_damping_0.5.json'
Results file maybe empty, cannot be loaded.
Loaded 0 results from output/ircot_v2/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.8_damping_0.5.json


IRCoT retrieval: 100%|██████████| 1000/1000 [02:50<00:00,  5.88it/s]


Saved 1000 results to output/ircot_v2/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.8_damping_0.5.json


CompletedProcess(args=['python3', 'src/ircot_hipporag_v2.py', '--dataset', 'musique', '--retriever', 'facebook/contriever', '--max_steps', '1', '--doc_ensemble', 'f', '--top_k', '10', '--sim_threshold', '0.8', '--damping', '0.5'], returncode=0)

In [7]:
calculate_recall_mean('output/ircot_v2/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_no_ensemble_step_1_top_10_sim_thresh_0.8_damping_0.5.json')

recall@1      0.275326
recall@2      0.370912
recall@5      0.465914
recall@10     0.526998
recall@15     0.526998
recall@20     0.526998
recall@30     0.526998
recall@40     0.526998
recall@50     0.526998
recall@80     0.526998
recall@100    0.526998
dtype: float64

In [8]:
# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

create_graph_command = [
    'python3', 'src/create_graph_v2.py',
    '--dataset', 'musique',
    '--model_name', 'facebook/contriever',
    '--extraction_model', 'gpt-3.5-turbo-1106',
    '--threshold', "0.8",
    '--extraction_type', 'ner',
    '--create_graph', '--cosine_sim_edges'
]
subprocess.run(create_graph_command, check=True)

ircot_hipporag_command = [
    'python3', 'src/ircot_hipporag_v2.py',
    '--dataset', 'musique',
    '--retriever', 'facebook/contriever',
    '--max_steps', '1',
    '--doc_ensemble', 't',
    '--top_k', '10',
    '--sim_threshold', "0.8",
    '--damping', '0.5',
]
subprocess.run(ircot_hipporag_command, check=True)


100%|██████████| 11656/11656 [00:01<00:00, 10467.01it/s]


Correct Wiki Format: 0 out of 11656
Creating Graph


100%|██████████| 11656/11656 [00:00<00:00, 36120.55it/s]


Loading Vectors
Augmenting Graph from Similarity


100%|██████████| 113611/113611 [00:03<00:00, 30887.62it/s]


...

Building Graph: 100%|██████████| 585007/585007 [00:00<00:00, 590957.43it/s]
2024-07-12 15:43:27,176 - hipporag_v2 - INFO - Graph built: num vertices: 113611, num_edges: 584896
2024-07-12 15:43:27,220 - hipporag_v2 - INFO - Loading node vectors from: data/lm_vectors/facebook_contriever_mean/encoded_strings.txt
2024-07-12 15:43:27,918 - hipporag_v2 - INFO - 1 phrases did not have vectors.
2024-07-12 15:43:27,954 - hipporag_v2 - INFO - Loaded doc embeddings from data/lm_vectors/facebook_contriever_mean/musique_doc_embeddings.p, shape: (11656, 768)
IRCoT retrieval:   0%|          | 0/1000 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'output/ircot_v2/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.8_damping_0.5.json'
Results file maybe empty, cannot be loaded.
Loaded 0 results from output/ircot_v2/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.8_damping_0.5.json


IRCoT retrieval: 100%|██████████| 1000/1000 [02:57<00:00,  5.62it/s]


Saved 1000 results to output/ircot_v2/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.8_damping_0.5.json


CompletedProcess(args=['python3', 'src/ircot_hipporag_v2.py', '--dataset', 'musique', '--retriever', 'facebook/contriever', '--max_steps', '1', '--doc_ensemble', 't', '--top_k', '10', '--sim_threshold', '0.8', '--damping', '0.5'], returncode=0)

In [9]:
calculate_recall_mean('output/ircot_v2/ircot_results_musique_hipporag_facebook_contriever_demo_1_gpt-3_5-turbo-1106_doc_ensemble_0.9_step_1_top_10_sim_thresh_0.8_damping_0.5.json')

recall@1      0.296491
recall@2      0.409497
recall@5      0.526917
recall@10     0.622585
recall@15     0.622585
recall@20     0.622585
recall@30     0.622585
recall@40     0.622585
recall@50     0.622585
recall@80     0.622585
recall@100    0.622585
dtype: float64